In [ ]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
import os
import sklearn
# from math import pi
# from matplotlib.path import Path
# from matplotlib.spines import Spine
# from matplotlib.transforms import Affine2D
from tensorflow.keras.models import load_model
import sys
from numpy import dot
from numpy.linalg import norm
import pickle

In [ ]:
tf.config.set_visible_devices([], 'GPU')

In [ ]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def cos_sim_class(vec_dic, label):          
    df_label = name[name['class_label']==label].reset_index(drop=True)
    
    sim_label = []

    for i in tqdm_notebook(range(len(df_label))):
        sim_label.append(cos_sim(np.squeeze(vec_dic[df_label['DrugA'][i]]), np.squeeze(vec_dic[df_label['DrugB'][i]])))
    
    return sim_label

def show_sim_aver(sim_df):
    group_1 = []
    group_2 = []
    group_3 = []
    group_4 = []
    idx = 0
    for i in sim_df['DDI type'].unique():
        group_1.append(len(sim_df[(sim_df['DDI type']==i) & (sim_df['Similarity'] > 0) & (sim_df['Similarity'] <=0.3)]))
        group_2.append(len(sim_df[(sim_df['DDI type']==i) & (sim_df['Similarity'] > 0.3) & (sim_df['Similarity'] <=0.5)]))
        group_3.append(len(sim_df[(sim_df['DDI type']==i) & (sim_df['Similarity'] > 0.5) & (sim_df['Similarity'] <=0.7)]))
        group_4.append(len(sim_df[(sim_df['DDI type']==i) & (sim_df['Similarity'] > 0.7) & (sim_df['Similarity'] <=1)]))
        idx = idx + 1
        
    print('0 to 0.3: {}'.format(np.mean(group_1)))
    print('0.3 to 0.5: {}'.format(np.mean(group_2)))
    print('0.5 to 0.7: {}'.format(np.mean(group_3)))
    print('0.7 to 1: {}'.format(np.mean(group_4)))
    return idx, group_1

def get_model(model_path):
    model_best = load_model(model_path, compile=False)
    model_best.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])
    
    return model_best

model_path = '../Model/Model_save/model_final.h5'
test_set = np.load('../Generated_data/test_set.npz')

test_A = test_set['test_A']
test_B = test_set['test_B']

one_hot_test = pd.read_csv('../Generated_data/one_hot_test.csv', index_col=0)

temp = one_hot_test.reset_index(drop=True)

label = list(one_hot_test.columns)

y_test = []

for i in tqdm_notebook(range(len(temp))):
    y_test.append(int(label[list(temp.iloc[i]).index(1)]))
    
model_best = get_model(model_path)

cls = list(one_hot_test.columns)
lay = [18,19]

# testset index로 데이터셋 구성
name = pd.read_csv('../Data/base_df.csv')
name = name[['DrugA', 'DrugB', 'class_label']].iloc[list(one_hot_test.index)].reset_index(drop=True)
    
#중복 제거 후 약물 인덱스 추출
drug_A = name.drop_duplicates(['DrugA'])
drug_B = name.drop_duplicates(['DrugB'])
    
drug_A_idx = list(drug_A.index)
drug_B_idx = list(drug_B.index)

t = tf.keras.Model(inputs = model_best.inputs, outputs = [model_best.layers[lay[0]].output, model_best.layers[lay[1]].output])
    
drug_vec = t.predict({'inputA':test_A, 'inputB': test_B})

vec_dic = {}

for i in tqdm_notebook(drug_A_idx):
    vec_dic[drug_A['DrugA'][i]] = drug_vec[0][i]
    
for i in tqdm_notebook(drug_B_idx):
    if drug_B['DrugB'][i] not in vec_dic:
        vec_dic[drug_B['DrugB'][i]] = drug_vec[1][i]
        
mod = sys.modules[__name__]
sim_list = []

for cl in cls:
    sim_list.append(cos_sim_class(vec_dic, int(cl)))
    
sim_df = pd.DataFrame(columns = ['Similarity', 'Label'])

for i in range(len(sim_list)):
    la_li = []
    sim_li = sim_list[i]
    for j in range(len(sim_li)):
        la_li.append(int(cls[i]))
        
    temp_df = pd.DataFrame({'Similarity':sim_li, 'Label':la_li})
    sim_df = pd.concat([sim_df, temp_df], axis = 0)
    
sim_df = sim_df.rename(columns = {'Label':'DDI type'})

for i in tqdm_notebook(range(len(name))):
    name.loc[i, 'Similarity'] = cos_sim(np.squeeze(vec_dic[name['DrugA'][i]]), np.squeeze(vec_dic[name['DrugB'][i]]))
    
sns.set()
sns.set_context("talk", rc={"font.family":'Arial'}, font_scale = 0.7)
ax = sns.stripplot(data = sim_df, x = 'DDI type', y = 'Similarity', size = 3, palette = 'deep')
ax.set_ylabel('Similarity', weight = 'bold')
ax.set_xlabel('DDI type', weight = 'bold')

plt.show()